In [4]:

import numpy as np
path = "/home/vml/deepmm_ws/ASE/ase/poselib/data/"
npy_file = "cml@userMotion2.npy"
npy_file = 'retargeted/cml@m5_slow.npy'
# path = "/home/vml/deepmm_ws/isaac_ws/isaac_example/ASE/ase/data/motions/"
# npy_file = "amp_humanoid_walk.npy"

motion_dict = np.load(path + npy_file, allow_pickle=True).item()

print(motion_dict['rotation']['arr'].shape)
print(motion_dict.keys())

(300, 15, 4)
odict_keys(['rotation', 'root_translation', 'global_velocity', 'global_angular_velocity', 'skeleton_tree', 'is_local', 'fps', '__name__'])


In [47]:
import torch
def quat_mul(a, b):
    assert a.shape == b.shape
    shape = a.shape
    a = a.reshape(-1, 4)
    b = b.reshape(-1, 4)

    x1, y1, z1, w1 = a[:, 0], a[:, 1], a[:, 2], a[:, 3]
    x2, y2, z2, w2 = b[:, 0], b[:, 1], b[:, 2], b[:, 3]
    ww = (z1 + x1) * (x2 + y2)
    yy = (w1 - y1) * (w2 + z2)
    zz = (w1 + y1) * (w2 - z2)
    xx = ww + yy + zz
    qq = 0.5 * (xx + (z1 - x1) * (x2 - y2))
    w = qq - ww + (z1 - y1) * (y2 - z2)
    x = qq - xx + (x1 + w1) * (x2 + w2)
    y = qq - yy + (w1 - x1) * (y2 + z2)
    z = qq - zz + (z1 + y1) * (w2 - x2)

    quat = torch.stack([x, y, z, w], dim=-1).view(shape)

    return quat

In [48]:
pelvis_rot = torch.tensor(motion_dict['rotation']['arr'][:, 0, :])
right_upper_arm_rot = torch.tensor(motion_dict['rotation']['arr'][:, 3, :])
left_upper_arm_rot = torch.tensor(motion_dict['rotation']['arr'][:, 6, :])

In [51]:
motion_dict['rotation']['arr'][:, 3, :] = quat_mul(right_upper_arm_rot, pelvis_rot).numpy() # right arm
motion_dict['rotation']['arr'][:, 6, :] = quat_mul(left_upper_arm_rot, pelvis_rot).numpy() # left arm
motion_dict['rotation']['arr'][:, 0, :] = np.array([0.0, 0.0, 0.0, 1.0])

In [52]:
path = "/home/vml/deepmm_ws/ASE/ase/poselib/data/"
npy_file = 'retargeted/cml@m5_slow_root_fixed.npy'

np.save(path+npy_file, motion_dict)